### @khan mustuffa Dated 13/05/2022

In [3]:
#load required Libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [7]:
#checking for the device
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [55]:
#transforms
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1 numpy to tensors
    transforms.Normalize([0.5,0.5,0.5],#0-1 to [-1,1], fromula = (x-mean)/std
                        [0.5,0.5,0.5])
])


In [56]:
#Data_Loaders
#Paths to train and test folders
train_path ='retina_lesion_/train'
test_path  = 'retina_lesion_/test'

train_loader = DataLoader(
   torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size = 32, shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets. ImageFolder(test_path, transform= transformer),
    batch_size  =32, shuffle =True
)

In [57]:
#categories
root = pathlib.Path(train_path)
classes =sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['class0', 'class1', 'class2', 'class3', 'class4']

In [58]:
#Building Neural Network
class ConvNet(nn.Module):
    def __init__(self, num_classes = 5):
        super(ConvNet, self).__init__()
        
        #input shape= (512, 3, 512, 512)
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels=12,kernel_size = 3, stride = 1, padding=1)
        
        #shape =(512, 12, 512, 512)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1 = nn.ReLU()
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        self.pool = nn.MaxPool2d(kernel_size = 2)
        
        self.Conv2  = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3, stride =1, padding=1)
        self.relu2 = nn.ReLU()
        
        self.Conv3  =nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride = 1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        self.relu3  =nn.ReLU()
        
        self.fc = nn.Linear(in_features =32*75*75, out_features = num_classes)

    #feed forward
    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)

        output= self.pool(output)

        output= self.Conv2(output)
        output= self.relu2(output)

        output= self.Conv3(output)
        output= self.bn3(output)
        output= self.relu3(output)

        output=output.view(-1, 32*75*75)
        output=self.fc(output)

        return output

        

In [59]:
model = ConvNet(num_classes=5).to(device)

In [60]:
#optimizer and loss function
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay=0.0001)
loss_function= nn.CrossEntropyLoss()

In [64]:
num_epochs =50

In [65]:
#calculating the size of training and testing images
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count  = len(glob.glob(test_path+'/**/*.jpg'))
print(train_count, test_count)

1373 158


In [66]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

Epoch: 0 Train Loss: tensor(0.8535) Train Accuracy: 0.7042971595047341 Test Accuracy: 0.22784810126582278
Epoch: 1 Train Loss: tensor(0.7635) Train Accuracy: 0.71959213401311 Test Accuracy: 0.5949367088607594
Epoch: 2 Train Loss: tensor(0.5379) Train Accuracy: 0.8077203204661325 Test Accuracy: 0.45569620253164556
Epoch: 3 Train Loss: tensor(0.3631) Train Accuracy: 0.8812818645302258 Test Accuracy: 0.5063291139240507
Epoch: 4 Train Loss: tensor(0.4803) Train Accuracy: 0.831755280407866 Test Accuracy: 0.5949367088607594
Epoch: 5 Train Loss: tensor(0.4838) Train Accuracy: 0.8259286234522942 Test Accuracy: 0.5632911392405063
Epoch: 6 Train Loss: tensor(0.4097) Train Accuracy: 0.8557902403495994 Test Accuracy: 0.5379746835443038
Epoch: 7 Train Loss: tensor(0.2470) Train Accuracy: 0.9184268026219956 Test Accuracy: 0.5
Epoch: 8 Train Loss: tensor(0.1478) Train Accuracy: 0.9555717407137655 Test Accuracy: 0.5063291139240507
Epoch: 9 Train Loss: tensor(0.1075) Train Accuracy: 0.9694100509832484 